In [1]:
!pip install xgboost

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor

In [3]:
from google.colab import files
uploaded = files.upload()

Saving hospital_dataset.csv to hospital_dataset.csv
Saving aqi_dataset.csv to aqi_dataset.csv
Saving climate_dataset.csv to climate_dataset.csv
Saving raw_weekly_hospital_respiratory_data_2020_2024.csv to raw_weekly_hospital_respiratory_data_2020_2024.csv


In [4]:
import pandas as pd
import numpy as np

climate = pd.read_csv("climate_dataset.csv")
aqi = pd.read_csv("aqi_dataset.csv")
synthetic_hospital = pd.read_csv("hospital_dataset.csv")
raw_hospital = pd.read_csv("raw_weekly_hospital_respiratory_data_2020_2024.csv")

In [5]:
raw_hospital.head()

,Week Ending Date,Geographic aggregation,Number of Inpatient Beds,Number of Adult Inpatient Beds,Number of Pediatric Inpatient beds,Number of Inpatient Beds Occupied,Number of Adult Inpatient Beds Occupied,Number of Pediatric Inpatient Beds Occupied,Number of ICU Beds,Number of Adult ICU Beds,...,Absolute Change in the Percent Hospitals Reporting Adult RSV Admissions from Prior Week,Absolute Change in the Percent Hospitals Reporting RSV Admissions from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Inpatient Beds Occupied from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent ICU Beds Occupied from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Inpatient Beds Occupied by COVID-19 Patients from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Inpatient Beds Occupied by Influenza Patients from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent Inpatient Beds Occupied by RSV Patients from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent ICU Beds Occupied by COVID-19 Patients from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent ICU Beds Occupied by Influenza Patients from Prior Week,Absolute Change in the Percent Hospitals Reporting Percent ICU Beds Occupied by RSV Patients from Prior Week
0,2023-08-12,WA,10650.0,9282.0,482.0,9040.0,8117.0,339.0,1623.0,1059.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
1,2023-08-12,WI,10752.0,9824.0,425.0,7574.0,7027.0,286.0,1566.0,1232.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
2,2023-08-12,WV,4813.0,4556.0,256.0,3701.0,3562.0,139.0,639.0,595.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0
3,2023-08-12,WY,1318.0,1142.0,25.0,559.0,534.0,6.0,114.0,109.0,...,0.0,0.0,0.03,0.03,0.03,0.03,0.0,0.03,0.03,0.0
4,2023-08-19,AK,1354.0,1192.0,77.0,1054.0,939.0,43.0,184.0,126.0,...,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.0


In [6]:
# Convert date
raw_hospital["Week Ending Date"] = pd.to_datetime(raw_hospital["Week Ending Date"])

# Rename
raw_hospital.rename(columns={
    "Week Ending Date": "date",
    "Geographic aggregation": "region"
}, inplace=True)

# Create total admissions
raw_hospital["actual_admissions"] = (
    raw_hospital["Total COVID-19 Admissions"].fillna(0) +
    raw_hospital["Total Influenza Admissions"].fillna(0) +
    raw_hospital["Total RSV Admissions"].fillna(0)
)

# Keep only necessary columns
real_hospital = raw_hospital[[
    "date",
    "region",
    "actual_admissions",
    "Percent Inpatient Beds Occupied",
    "Percent ICU Beds Occupied"
]]

real_hospital.head()

,date,region,actual_admissions,Percent Inpatient Beds Occupied,Percent ICU Beds Occupied
0,2023-08-12,WA,245.0,0.85,0.81
1,2023-08-12,WI,148.0,0.70,0.65
2,2023-08-12,WV,54.0,0.77,0.73
3,2023-08-12,WY,13.0,0.42,0.35
4,2023-08-19,AK,22.0,0.78,0.76


In [11]:
# Ensure 'date' is a column if it's currently an index
# This makes the cell re-runnable even if 'date' was already set as index previously.
if 'date' in real_hospital.index.names:
    real_hospital = real_hospital.reset_index(level='date')
elif real_hospital.index.name == 'date':
    real_hospital = real_hospital.reset_index()

# Now 'date' should be a column.
# Set 'date' as the index before grouping and resampling.
real_hospital = real_hospital.set_index('date')

# Identify numerical columns to interpolate. Exclude 'region' as it's a string and should not be interpolated.
numerical_cols = real_hospital.select_dtypes(include=np.number).columns.tolist()

# Group by region, then resample daily and interpolate only the numerical columns within each group.
# The 'region' column will naturally become an index level due to groupby, and then a column again on reset_index()
# without conflict because it was not part of the columns being interpolated directly.
real_hospital_daily = (
    real_hospital.groupby('region')[numerical_cols] # Selects only numerical columns for the grouped operation
    .resample('D')
    .interpolate()
    .reset_index()
)

real_hospital_daily.head()

,region,date,actual_admissions,Percent Inpatient Beds Occupied,Percent ICU Beds Occupied
0,AK,2020-08-08,49.000000,0.680000,0.490000
1,AK,2020-08-09,48.857143,0.681429,0.497143
2,AK,2020-08-10,48.714286,0.682857,0.504286
3,AK,2020-08-11,48.571429,0.684286,0.511429
4,AK,2020-08-12,48.428571,0.685714,0.518571


In [21]:
climate["date"] = pd.to_datetime(climate["date"], errors='coerce')
aqi["date"] = pd.to_datetime(aqi["date"], errors='coerce')

# Clean region names by stripping whitespace to ensure proper merging
climate['region'] = climate['region'].str.strip()
aqi['region'] = aqi['region'].str.strip()

# Drop rows where merge keys might be invalid (NaT for date, or empty strings for region)
climate = climate.dropna(subset=['date', 'region'])
aqi = aqi.dropna(subset=['date', 'region'])

data = pd.merge(climate, aqi, on=["date", "region"], how="inner")

In [24]:
synthetic_hospital["date"] = pd.to_datetime(synthetic_hospital["date"], errors='coerce')

data = pd.merge(data, synthetic_hospital, on=["date", "region"], how="inner")

# Drop rows where merge keys might be invalid in synthetic_hospital data as well
synthetic_hospital = synthetic_hospital.dropna(subset=['date', 'region'])

data = data.dropna()
data.head()

,date,region,avg_temp,humidity,heat_index,aqi,actual_admissions,percent_beds_occupied,percent_icu_occupied,prev_admissions
0,2010-01-02,Bangalore,31.044318,45.403299,35.584648,66.872558,169.581163,68.479058,56.783247,141.322922
1,2010-01-03,Bangalore,25.803912,61.907876,31.994700,133.250537,193.314207,69.665710,57.732568,169.581163
2,2010-01-04,Bangalore,30.435235,62.009926,36.636228,123.670431,152.940449,67.647022,56.117618,193.314207
3,2010-01-05,Bangalore,31.480601,67.610553,38.241657,120.204538,179.763457,68.988173,57.190538,152.940449
4,2010-01-06,Bangalore,29.826441,53.313220,35.157763,165.712913,205.732531,70.286627,58.229301,179.763457


In [26]:
# Heat index already exists
data["prev_admissions"] = data.groupby("region")["actual_admissions"].shift(1)

data = data.dropna()

/tmp/ipython-input-1610682598.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["prev_admissions"] = data.groupby("region")["actual_admissions"].shift(1)


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor

X = data[[
    "avg_temp",
    "humidity",
    "heat_index",
    "aqi",
    "percent_beds_occupied", # Changed from "Percent Inpatient Beds Occupied"
    "percent_icu_occupied",  # Changed from "Percent ICU Beds Occupied"
    "prev_admissions"
]]

y = data["actual_admissions"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = XGBRegressor(n_estimators=400, learning_rate=0.05, max_depth=6)
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print("R2:", r2_score(y_test, predictions))
print("RMSE:", np.sqrt(mean_squared_error(y_test, predictions)))

R2: 0.9995194587531266
RMSE: 0.4437244800927829


In [30]:
import pandas as pd

# Create your custom input
test_input = pd.DataFrame({
    "avg_temp": [38],   # High temperature
    "humidity": [75],   # High humidity
    "heat_index": [38 + 0.1*75],
    "aqi": [180],       # Poor air quality
    "percent_beds_occupied": [80], # Corrected column name
    "percent_icu_occupied": [70],  # Corrected column name
    "prev_admissions": [220]
})

test_input

,avg_temp,humidity,heat_index,aqi,percent_beds_occupied,percent_icu_occupied,prev_admissions
0,38,75,45.5,180,80,70,220


In [31]:
prediction = model.predict(test_input)

print("Predicted Admissions:", prediction[0])

Predicted Admissions: 245.51189


In [32]:
baseline = data["actual_admissions"].mean()

percent_increase = ((prediction[0] - baseline) / baseline) * 100

print("Percent Increase:", percent_increase)

Percent Increase: 35.57643943466896


In [33]:
total_beds = 1000   # Example capacity

overload_probability = prediction[0] / total_beds

print("Overload Probability:", overload_probability)

Overload Probability: 0.24551189


In [34]:
if percent_increase < 10:
    risk = "LOW"
elif percent_increase < 20:
    risk = "MODERATE"
else:
    risk = "HIGH"

print("Risk Level:", risk)

Risk Level: HIGH


In [37]:
multiple_test = pd.DataFrame({
    "avg_temp": [30, 35, 42],
    "humidity": [60, 70, 85],
    "heat_index": [30+6, 35+7, 42+8.5],
    "aqi": [100, 150, 220],
    "percent_beds_occupied": [65, 75, 90], # Corrected column name
    "percent_icu_occupied": [55, 65, 85],  # Corrected column name
    "prev_admissions": [150, 200, 300]
})

predictions = model.predict(multiple_test)

print(predictions)

[117.99882 241.97224 241.12471]


In [38]:
# Make prediction
prediction = model.predict(test_input)[0]

# Calculate baseline
baseline = data["actual_admissions"].mean()

# Percent increase
percent_increase = ((prediction - baseline) / baseline) * 100

# Overload probability (assuming total beds)
total_beds = 1000
overload_probability = prediction / total_beds

# Risk level classification
if percent_increase < 10:
    risk = "LOW"
elif percent_increase < 20:
    risk = "MODERATE"
else:
    risk = "HIGH"

# Final formatted output
print("Predicted Admissions:", round(prediction, 2))
print("Percent Increase:", round(percent_increase, 2))
print("Overload Probability:", round(overload_probability, 2))
print("Risk Level:", risk)

Predicted Admissions: 245.51
Percent Increase: 35.58
Overload Probability: 0.25
Risk Level: HIGH


In [39]:
import joblib

# Save trained model
joblib.dump(model, "climate_surge_model.pkl")

print("Model saved successfully!")

Model saved successfully!
